In [6]:
import torch
from torchvision.datasets import MNIST

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt

In [15]:
training_data = MNIST("../datasets/", train=True, download=True)
test_data = MNIST("../datasets/", train=False, download=True)

In [18]:
training_data.data.shape, training_data.targets.shape

(torch.Size([60000, 28, 28]), torch.Size([60000]))

In [20]:
from random import randint